In [1]:
from aocd import get_data
import numpy as np

day16 = get_data(day=16,year=2021)

This seems to require a function that can call itself.

In [7]:
def read_packet(bits):
    n = 0
    version = int(bits[n:n+3],base=2)
    versions.append(version)
    typeid = int(bits[n+3:n+6],base=2)
#    print('Version ',version,' Type ID ',typeid)
    n += 6
    if typeid==4:
        literal = ''
        leading = int(bits[n:n+1],base=2)
        literal += bits[n+1:n+5]
        n += 5
        while leading == 1:
            leading = int(bits[n:n+1],base=2)
            literal += bits[n+1:n+5]
            n += 5
#        print('Literal value ',int(literal,base=2),', packet length',n)
        return n
    else:
        lengthtypeid = int(bits[n:n+1],base=2)
        n += 1
        if lengthtypeid == 0:
            sublength = int(bits[n:n+15],base=2)
#            print('Operator with ',sublength,' bits of subpackets')
            n += 15
            l = 0
            while l < sublength:
                l += read_packet(bits[n+l:])
            n += l
        else:
            nsubpackets = int(bits[n:n+11],base=2)
#            print('Operator with ',nsubpackets,' subpackets')
            n += 11                
            for i in range(nsubpackets):
                n += read_packet(bits[n:])
    return n
        

In [8]:
bits = ''
for i in '8A004A801A8002F478':
    bits += bin(int(i,base=16))[2:].zfill(4)

versions = []
read_packet(bits)
print(np.sum(versions))

16


In [9]:
bits = ''
for i in day16:
    bits += bin(int(i,base=16))[2:].zfill(4)


versions = []
read_packet(bits)
print(np.sum(versions))

854


Got it!

In [95]:
def read_packet(bits):
    n = 0
    version = int(bits[n:n+3],base=2)
    typeid = int(bits[n+3:n+6],base=2)
    n += 6
    if typeid==4:
        literal = ''
        leading = int(bits[n:n+1],base=2)
        literal += bits[n+1:n+5]
        n += 5
        while leading == 1:
            leading = int(bits[n:n+1],base=2)
            literal += bits[n+1:n+5]
            n += 5
        ops.append(str(int(literal,base=2)))
        ops.append(',')
        return n
    else:
        if typeid == 0:
            ops.append('np.sum([')
        elif typeid == 1:
            ops.append('np.product([')
        elif typeid == 2:
            ops.append('np.min([')
        elif typeid == 3:
            ops.append('np.max([')
        elif typeid == 5:
            ops.append('np.greater(')
        elif typeid == 6:
            ops.append('np.less(')
        elif typeid == 7:
            ops.append('np.equal(')
        lengthtypeid = int(bits[n:n+1],base=2)
        n += 1
        if lengthtypeid == 0:
            sublength = int(bits[n:n+15],base=2)
            n += 15
            l = 0
            while l < sublength:
                l += read_packet(bits[n+l:])
                ops.append(',')
            n += l
        else:
            nsubpackets = int(bits[n:n+11],base=2)
            n += 11                
            for i in range(nsubpackets):
                n += read_packet(bits[n:])
                ops.append(',')
        if typeid < 4:
            ops.append(']')
        ops.append(')')
    return n
        

In [98]:
bits = ''
for i in '880086C3E88112':
    bits += bin(int(i,base=16))[2:].zfill(4)

ops = []
read_packet(bits)


55

In [99]:
eval(''.join(ops).replace(',,',',').replace(',]',']').replace(',)',')'))

7

In [100]:
bits = ''
for i in day16:
    bits += bin(int(i,base=16))[2:].zfill(4)

ops = []
read_packet(bits)   
print(eval(''.join(ops).replace(',,',',').replace(',]',']').replace(',)',')')))

181894873364


In [101]:
''.join(ops).replace(',,',',').replace(',]',']').replace(',)',')')

'np.sum([np.product([9,np.less(90,258)]),np.max([14,2]),np.min([15083,4,10]),np.min([46486,946644]),47731,np.product([86,167,76,41,75]),np.product([902426,np.less(5448597,5448597)]),12,3150483273,46921,1756,np.product([3098,np.equal(241,11104915)]),np.product([np.greater(2338,2338),185954532]),np.product([105,146,64,124]),1619,np.product([183,np.less(440674,103)]),np.max([6937079,92140,209,592]),np.max([np.product([np.min([np.min([np.max([np.sum([np.max([np.product([np.max([np.sum([np.min([np.sum([np.sum([np.min([np.max([np.sum([np.max([np.product([np.min([np.min([5056208039])])])])])])])])])])])])])])])])])])])]),455,256083,np.sum([np.product([9,6,4]),np.product([14,4,8]),np.product([5,3,12])]),np.min([11,208,70,1647,130]),np.product([3586,np.greater(np.sum([14,2,7]),np.sum([5,6,3]))]),np.product([1045,np.equal(7539326,139)]),np.product([28505,np.less(1344,46)]),np.product([np.equal(310846,310846),55133606844]),2158747088,np.max([50614281761]),np.product([44]),np.product([np.sum([10,1

In [85]:
''.join(ops)

'np.equal(np.sum([1,,3,,]),np.product([2,,2,,]),)'